In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.plotting.backend = "plotly"

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
input_folder='/kaggle/input/jane-street-market-prediction/'
training_path = input_folder+'train.csv'
features_path = input_folder+'features.csv'
testing_path = input_folder+'example_test.csv'

In [ ]:
!pip install datatable
import datatable as dt
import matplotlib.pyplot as plt
import seaborn as sns  
import plotly.offline as py

import plotly.express as px
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.subplots import make_subplots



Load the dataframes

In [ ]:

train_df_full = dt.fread(training_path).to_pandas()

# test_df = dt.fread(testing_path).to_pandas()
# features_df = dt.fread(features_path).to_pandas()

Let's see the shapes of the respectives dataframes

In [ ]:
save = 'Testing'
if save == 'Testing':
    train_df = train_df_full.head(10000)

In [ ]:
print('Training dataframe')
print(f'Number of Columns : {len(train_df.columns)}')
print(f'Number of Rows: {len(train_df.index)}\n')

# print('Features dataframe shape')
# print(f'Number of Columns : {len(features_df/.columns)}')
# print(f'Number of Rows: {len(features_df.index)}\n')

# print('Testing dataframe')
# print(f'Number of Columns : {len(test_df.columns)}')
# print(f'Number of Rows: {len(test_df.index)}\n')

Define Action for the dataset

In [ ]:
actn = ((train_df['weight'].values * train_df['resp'].values) > 0).astype('int')
train_df = train_df.assign(action = actn)


Let's have a look at the training dataset

In [ ]:
train_df.head()

In [ ]:
# test_df.head()

In [ ]:
train_df.describe()

In [ ]:
def count_missing_values(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return tt

In [ ]:
train_missing  = count_missing_values(train_df)
train_missing = train_missing[train_missing.Total !=0]

bars=go.Bar(x=train_missing.index.values,y=train_missing.Total,name = 'Total')
line = go.Scatter(x=train_missing.index.values,y=train_missing.Percent,name = 'Percent')
fig = make_subplots(specs=[[{"secondary_y": True}]], print_grid=True)
fig.add_trace(bars, 1, 1, secondary_y=False)
fig.add_trace(line, 1, 1, secondary_y=True)
fig.update_layout(
    title="Missing Values",
    xaxis_title="Column Names",
    yaxis_title="Number of Missing Values",
    legend_title="Legend",
    
)

fig.show()


Columns with more than 25% missing data

In [ ]:
columns_missing = []
for idx,i in train_missing['Percent'].iteritems():
 
    if i > 5:
        columns_missing.append([idx, i])
        
print(*columns_missing,sep='\n')

In [ ]:
resp = []
subset = train_df[train_df.columns[0:7]]

for col in subset.columns[2:]:
#     resp.append(go.Scatter(
#         x=pd.Series(train_df[col].cumsum()), name = col
#         ))
    resp.append(go.Scatter(y=pd.Series(subset[col].cumsum()),
        name = col
    ))
layout = go.Layout(dict(title = "Cummulative sum of responses",
                  yaxis = dict(title = 'Response'),
                  ),legend=dict(
                orientation="h"))
        
py.iplot(dict(data=resp, layout=layout), filename='basic-line')   


HeatMap


In [ ]:
def df_to_heatmap(df):
    
    return {       'y': df.columns.values,
            'x': df.columns.values,
           'z':df.corr().values}

In [ ]:
train_df_test = train_df.head(10)

figure = go.Figure(go.Heatmap(df_to_heatmap(train_df_test)))
figure.show()

In [ ]:
fig  = px.bar(train_df['action'].value_counts(),x='action', color='action')
fig.show()

Violin and Histograms of all the Data

In [ ]:


fig = make_subplots(rows=len(train_df.columns), cols=2,vertical_spacing = 0.5/len(train_df.columns))

color1 = '#9467bd'
color2 = '#F08B00'


In [ ]:
for idx,col in enumerate(train_df.columns):
#     print(idx,col)
    fig.add_trace( go.Scatter(
        y = train_df[col],
        name=col,
        mode = 'markers',
        line = dict(
            color = color1
        )),row = idx+1, col =1)
    fig.add_trace( go.Violin(y=train_df[col],name=col),row=idx+1,col=2)


In [ ]:
fig['layout'].update(height=100000)
fig.show()

In [ ]:
X_train = train_df.loc[:, train_df.columns.str.contains('feature')]
X = X_train.fillna(X_train.mean())
# our target is the action
y = train_df['action']
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42)


In [ ]:

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_features='auto')
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
preds = classifier.predict(X_valid)
print('Score: {}'.format(accuracy_score(preds, y_valid)))